# main

In [1]:
import json
import requests
import popelines
import os
from main import fix_values, populating_vals, flatten, flatten_dupe_vals
from datetime import datetime

In [2]:
def fix_values(value, key, reset_key):
    if key == reset_key:
        new_list = []
        for x in value:
            value[x][f'{reset_key}_id'] = x
            new_list.append(value[x])
        return new_list
    else:
        return value

In [3]:
# read endpoint, returns a json file of the HTTP request
def read_endpoint(endpoint, headers_set, params_set=None):
    try:
        response = requests.get(endpoint, headers=headers_set, params=params_set)
        response_text = json.loads(response.text)
        response.raise_for_status()

    except requests.exceptions.HTTPError as err:
        print(err)

    return response_text

In [4]:
# generate all experiments given a project id
def generate_experiments(project_id, experiment_endpoint, experiment_headers, experiment_params):
    # get all experiments from one project
    j_exp = read_endpoint(endpoint=experiment_endpoint, headers_set=experiment_headers, params_set=experiment_params)

    # loop for all experiments in this project
    experiment_id_list = []
    upload_exp_list = []
    for exp in j_exp:
        exp['project_id'] = project_id
        upload_exp_list.append(flatten(exp, {}, ''))
        experiment_id_list.append(exp['id'])
        
    return upload_exp_list, experiment_id_list

In [5]:
# generate all projects within account
def generate_projects(project_endpoint, project_headers):
    # get all projects
    j_proj = read_endpoint(endpoint=project_endpoint, headers_set=project_headers)

    # store a list of project metadata
    return j_proj

# load all projects, get project_id's

In [6]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')
pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)

# Optimizely parameters
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
    'Content-Type': 'application/json',
}

In [16]:
project_endpoint = 'https://api.optimizely.com/v2/projects'

In [17]:
all_projects = generate_projects(project_endpoint, headers)

In [ ]:
# pope.write_to_json(file_name='../uploads/projects.json', jayson=all_projects, mode='w')

In [ ]:
# pope.write_to_bq(table_name='projects', file_name='../uploads/projects.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
# print("Successfully uploaded all projects.")

In [18]:
project_id_list = []
for project in all_projects:
    project_id_list.append((project['id']))

In [ ]:
# project_id_list


In [ ]:
# project_id_name = []
# for project in all_projects:
#     project_id_name.append((project['id'], project['name']))

In [ ]:
# project_id_name

# load experiments

In [14]:
experiment_endpoint = 'https://api.optimizely.com/v2/experiments'


In [25]:
# experiment_id_list = []
# all_exp_list = []

# for project_id in project_id_list:
    # params include project_id (required) and experiments pulling per each request (default only 25)
params = (
    ('project_id', 9965963792),
    ('per_page', 100),
) 

exp_list, exp_id_list = generate_experiments(project_id, experiment_endpoint, headers, params)
# experiment_id_list.extend(exp_id_list)
# all_exp_list.extend(exp_list)
    
#     # upload experiments
#     pope.write_to_json(file_name='../uploads/experiments.json', jayson=exp_list, mode='w')
#     pope.write_to_bq(table_name='experiments', file_name='../uploads/experiments.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
#     print(f"Successfully uploaded experiments for project {project_id}")

In [29]:
j_exp = read_endpoint(endpoint=experiment_endpoint, headers_set=headers, params_set=params)



In [38]:
for k,v in j_exp[0].items():
    if isinstance(v, list) is True or isinstance(v, dict) is True:
        print(k, type(v))

changes <class 'list'>
metrics <class 'list'>
url_targeting <class 'dict'>
variations <class 'list'>


In [48]:
def is_list_nesting_dict(lst):
    if isinstance(lst[0], dict) is True:
        return True
    else:
        return False
    

In [51]:
is_list_nesting_dict(j_exp[0]['metrics'])

True

In [99]:
def if_expandable(file):
    if isinstance(file, list) is True:
        if file == []:
            return False
        else:
            return is_list_nesting_dict(file)
    elif isinstance(file, dict) is True:
        return file.keys()
    else:
        return False

In [100]:
if_expandable(j_exp[0])

dict_keys(['allocation_policy', 'audience_conditions', 'campaign_id', 'changes', 'created', 'description', 'earliest', 'holdback', 'id', 'is_classic', 'last_modified', 'latest', 'metrics', 'name', 'project_id', 'status', 'type', 'url_targeting', 'variations'])

In [47]:
j_exp[0]['url_targeting']

{'activation_type': 'url_changed',
 'conditions': '["and", ["or", {"match_type": "simple", "type": "url", "value": "freetrial.infusionsoft.com/setup"}, {"match_type": "simple", "type": "url", "value": "freetrial.infusionsoft.com/signup"}]]',
 'edit_url': 'freetrial.infusionsoft.com/setup',
 'key': '9965963792_url_targeting_for_ft_signup_variations',
 'page_id': 10752650977}

In [44]:
j_exp[0]['variations']

[{'actions': [{'changes': [{'async': False,
      'attributes': {'remove': True},
      'css': {},
      'dependencies': [],
      'id': '1E6E5051-B355-49D5-B7FF-758F15524D61',
      'rearrange': {'insertSelector': '', 'operator': 'before'},
      'selector': '.checkbox',
      'type': 'attribute'},
     {'async': False,
      'attributes': {'html': 'Start your free trial'},
      'css': {},
      'dependencies': [],
      'id': 'F3E2FE9D-A546-4748-8B3C-4E1F8A47C609',
      'rearrange': {'insertSelector': '', 'operator': 'before'},
      'selector': 'form h2',
      'type': 'attribute'},
     {'async': False,
      'attributes': {'html': '<span data-v-d77c2e22="" class="">\n            Start your 14-day free trial\n        </span>'},
      'css': {},
      'dependencies': [],
      'id': '672EFC3F-D6A6-4CEB-9D5C-925BEEE2B675',
      'rearrange': {'insertSelector': '', 'operator': 'before'},
      'selector': '.icon-button',
      'type': 'attribute'},
     {'async': False,
      'attri

In [ ]:
# loop for all experiments in this project
experiment_id_list = []
upload_exp_list = []
for exp in j_exp:
    flattened_exp = []
#     exp['project_id'] = project_id
    
    new_exp = populating_vals(outer_dict=metric, inner_flattened_list=flattened_results, destination_key='variations')
    flattened_metrics.extend(flatten_dupe_vals(vals=update_metrics, key='results'))
    
#     upload_exp_list.append(flatten(exp, {}, ''))
    experiment_id_list.append(exp['id'])

# results timeseries

In [ ]:
# experiment_id_list

In [7]:
# example of no results
# result_endpoint = 'https://api.optimizely.com/v2/experiments/11479333433/timeseries'
# example of normal time series
result_endpoint = 'https://api.optimizely.com/v2/experiments/11477653122/timeseries'
# example of 
# result_endpoint = f'https://api.optimizely.com/v2/experiments/{experiment_id}/timeseries'



In [8]:
response_ts = requests.get(result_endpoint, headers=headers)

# if '' then the experiment has not started yet
if response_ts.text == '':
    j_ts = {'experiment_id': experiment_id}
elif 'bad' in response_ts.text:
    j_ts = {'experiment_id': experiment_id}
else:
    j_ts = json.loads(response_ts.text)

new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')


In [9]:
flattened_j_ts = []

flattened_metrics = []
for metric in new_j_ts['metrics']:
    if 'results' in metric.keys():
        for ts in metric['results']:
            flattened_timeseries = []
            for element in ts['timeseries']:
                element['upload_ts'] = str(datetime.now())
                flattened_timeseries.append(flatten(element, {}, ''))

            # Replace old 'timeseries' with new 'flattened_timeseries'
            updated_results = populating_vals(outer_dict=ts, inner_flattened_list=flattened_timeseries, destination_key='timeseries')
            flattened_results = flatten_dupe_vals(vals=updated_results, key='timeseries')

        # Replace old 'metrics' with new 'flattened_results'
        update_metrics = populating_vals(outer_dict=metric, inner_flattened_list=flattened_results, destination_key='results')
        flattened_metrics.extend(flatten_dupe_vals(vals=update_metrics, key='results'))

    else:
        flattened_metrics = [flatten(new_j_ts, {}, '')]

update_new_j_ts = populating_vals(outer_dict=new_j_ts, inner_flattened_list=flattened_metrics, destination_key='metrics')
flattened_j_ts.extend(flatten_dupe_vals(vals=update_new_j_ts, key='metrics'))



In [13]:
j_ts

{'confidence_threshold': 0.9,
 'end_time': '2018-09-24T23:15:00.000000Z',
 'experiment_id': 11477653122,
 'metrics': [{'aggregator': 'unique',
   'event_id': 10736450755,
   'name': 'Interactive Demo Page',
   'results': {'11475874049': {'is_baseline': False,
     'level': 'variation',
     'name': 'Variation #1',
     'results_id': '11475874049',
     'timeseries': [{'lift': {'is_significant': False,
        'lift_status': 'equal',
        'significance': 0.0,
        'value': 0.0,
        'visitors_remaining': 9223372036854775807},
       'rate': 0.0,
       'samples': 2,
       'time': '2018-09-24T06:02:23.000000Z',
       'value': 0,
       'variance': 0.0},
      {'lift': {'is_significant': False,
        'lift_status': 'equal',
        'significance': 0.0,
        'value': 0.0,
        'visitors_remaining': 9223372036854775807},
       'rate': 0.0,
       'samples': 2,
       'time': '2018-09-24T06:12:42.570000Z',
       'value': 0,
       'variance': 0.0},
      {'lift': {'is_si

In [12]:
flattened_j_ts

[{'confidence_threshold': 0.9,
  'end_time': '2018-09-24T23:15:00.000000Z',
  'experiment_id': 11477653122,
  'metrics_aggregator': 'unique',
  'metrics_event_id': 10736450755,
  'metrics_name': 'Interactive Demo Page',
  'metrics_results_is_baseline': True,
  'metrics_results_level': 'variation',
  'metrics_results_name': 'Original',
  'metrics_results_results_id': '11554562367',
  'metrics_results_timeseries_rate': 0.0,
  'metrics_results_timeseries_samples': 1,
  'metrics_results_timeseries_time': '2018-09-24T06:02:23.000000Z',
  'metrics_results_timeseries_upload_ts': '2019-02-15 13:04:41.608906',
  'metrics_results_timeseries_value': 0,
  'metrics_results_timeseries_variance': 0.0,
  'metrics_results_variation_id': '11554562367',
  'metrics_scope': 'visitor',
  'metrics_winning_direction': 'increasing',
  'start_time': '2018-09-24T06:02:23.000000Z',
  'stats_config_confidence_level': 0.9,
  'stats_config_difference_type': 'relative',
  'stats_config_epoch_enabled': False},
 {'conf